In [1]:
from tempfile import tempdir
import SimpleITK as sitk
import numpy as np
import numpy.ma as ma
from matplotlib import pyplot
import matplotlib.pyplot as plt
import os
import sys
#import seaborn as sns
import pandas as pd
import radiomics

In [ ]:
pat_df = pd.read_csv("D:\\data\\Aaron\\ProstateMRL\\Data\\MRLPacks\\MeanValues\\1307.csv")

base_df = pat_df[pat_df["Fraction"].isin([1])]
base_df = base_df[base_df["Scan"].isin([1])]
print(base_df)
base_pros, base_glute, base_psoas = base_df.iloc[0]["ShrunkProstate"], base_df.iloc[0]["Glute"], base_df.iloc[0]["Psoas"]
print(base_pros, base_glute, base_psoas)

In [ ]:
mask_url = "D:\\data\\prostateMR_radiomics\\nifti\\SABR\\0001088\\MR6\\Masks\\0001088_MR6_shrunk_pros.nii"
image_url = "D:\\data\\prostateMR_radiomics\\nifti\\SABR\\0001088\\MR6\\BaseImages\\0001088_MR6_image.nii"

cropped_img, cropped_mask = get_cropped_image_specified_size(image_url, mask_url, [60,60,30])

In [6]:
pat = "0000653"
MR = "MR6"

In [14]:
g_mask_url = "D:\\data\\prostateMR_radiomics\\nifti\\SABR\\" + pat + "\\"+ MR + "\\Masks\\" + pat+ "_" + MR + "_glute.nii"
g_mask = sitk.ReadImage(g_mask_url)
g_mask_a = sitk.GetArrayFromImage(g_mask)
g_mask_a = g_mask_a/(np.max(g_mask_a))
print(np.max(g_mask_a))
#mask_a2 = ma.make_mask(mask_a)

p_mask_url = "D:\\data\\prostateMR_radiomics\\nifti\\SABR\\" + pat + "\\"+ MR + "\\Masks\\" + pat+ "_" + MR + "_psoas.nii"
p_mask = sitk.ReadImage(p_mask_url)
p_mask_a = sitk.GetArrayFromImage(p_mask)
p_mask_a = p_mask_a/(np.max(p_mask_a))
print(np.max(p_mask_a))

pr_mask_url = "D:\\data\\prostateMR_radiomics\\nifti\\SABR\\" + pat + "\\"+ MR + "\\Masks\\" + pat+ "_" + MR + "_shrunk_pros.nii"
pr_mask = sitk.ReadImage(pr_mask_url)
pr_mask_a = sitk.GetArrayFromImage(pr_mask)
pr_mask_a = pr_mask_a/(np.max(pr_mask_a))
print(np.max(pr_mask_a))


image = sitk.ReadImage("D:\\data\\prostateMR_radiomics\\nifti\\SABR\\" + pat + "\\"+ MR + "\\Reg-Raw\\" + pat+ "_" + MR + "_reg_img_3.nii")
#image = sitk.ReadImage("D:\\data\\prostateMR_radiomics\\nifti\\SABR\\" + pat + "\\"+ MR + "\\BaseImages\\" + pat+ "_" + MR + "_image.nii")
image_a = sitk.GetArrayFromImage(image)

#body_mask = sitk.ReadImage("D:\\data\\prostateMR_radiomics\\nifti\\SABR\\0001303\\MR5\\Masks\\0001303_MR5_body_mask.nii")
#body_mask_a = sitk.GetArrayFromImage(body_mask)


#image_a = image_a*body_mask_a

g_masked_image_a = ma.array(image_a, mask=np.logical_not(g_mask_a), keep_mask=False, hard_mask=True)
p_masked_image_a = ma.array(image_a, mask=np.logical_not(p_mask_a), keep_mask=False, hard_mask=True)
pr_masked_image_a = ma.array(image_a, mask=np.logical_not(pr_mask_a), keep_mask=False, hard_mask=True)
#g_image_ar2 = image_a * g_mask_a
#p_image_ar2 = image_a * p_mask_a

print(g_mask_a.shape)
print(p_mask_a.shape)
print(image_a.shape)

print(np.mean(image_a))  #Whole image
print("Glute: " + str(ma.mean(g_masked_image_a))) # masked image using ma module
#print(np.mean(g_image_ar2)) # just multiplying 
print("Psoas: " + str(ma.mean(p_masked_image_a))) # masked image using ma module
print("Pros: " + str(ma.mean(pr_masked_image_a)))
#print(np.mean(p_image_ar2)) # just multiplying 

1.0
1.0
1.0
(300, 480, 480)
(300, 480, 480)
(300, 480, 480)
49.70669554398148
Glute: 21.187713310580204
Psoas: 13.047244094488189
Pros: 54.44573245794009
54.44573245794009


In [11]:
def get_cropped_image_specified_size(datafilename, maskfilename, size = [0,0,0], im = True):
    image = datafilename
    gtv_gen = maskfilename
    
    im = sitk.ReadImage(image)
    
    #if im == True:
     #   im = sitk.Cast(sitk.IntensityWindowing(im, windowMinimum = 0, windowMaximum = 1500),
      #                 sitk.sitkUInt8)
    
    msk = sitk.ReadImage(gtv_gen)
    
    bb, corrMask = radiomics.imageoperations.checkMask(im,msk)
    
    dimx = bb[1] - bb[0]
    dimy = bb[3] - bb[2]
    dimz = bb[5] - bb[4]
    
    padDistancex = int((size[0] - dimx)/2)
    padDistancey = int((size[1] - dimy)/2)
    padDistancez = int((size[2] - dimz)/2)
    
    cropped_im, cropped_msk = cropToTumorMaskSquare(im,msk,bb,padDistancex,padDistancey,padDistancez)
    
    return cropped_im, cropped_msk

In [13]:
image_url = "D:\\data\\prostateMR_radiomics\\nifti\\SABR\\" + pat + "\\"+ MR + "\\Reg-Raw\\" + pat+ "_" + MR + "_reg_img_3.nii"
pr_mask_url = "D:\\data\\prostateMR_radiomics\\nifti\\SABR\\" + pat + "\\"+ MR + "\\Masks\\" + pat+ "_" + MR + "_shrunk_pros.nii"

cropped_img, cropped_mask = get_cropped_image_specified_size(image_url, pr_mask_url, [60,60,30])

ValueError: Image/Mask geometry mismatch. Potential fix: increase tolerance using geometryTolerance, see Documentation:Usage:Customizing the Extraction:Settings:geometryTolerance for more information

In [27]:
from datetime import datetime

d1 = "20120620"
d2 = "20130704"

#day1, day2 = int(d1[-2:]), int(d2[-2:])
def days_between(d1, d2):
    d1 = datetime.strptime(d1, "%Y%m%d")
    d2 = datetime.strptime(d2, "%Y%m%d")
    return abs((d2 - d1).days)

diff = days_between(d1,d2)
print(diff)


AttributeError: 'str' object has no attribute 'datatype'

In [26]:
date = "20120620"
d1 = datetime.strptime(date, "%Y%m%d")
print(d1.date())

2012-06-20


In [ ]:
patID = "1088"
patID_l = "0001088"

url_20f = 'D:/data/prostateMR_radiomics/nifti/20fractions/'
url_20f_new = 'D:/data/prostateMR_radiomics/nifti/20fractions_new/'
url_SABR = 'D:/data/prostateMR_radiomics/nifti/SABR/'
url_SABR_new = 'D:/data/prostateMR_radiomics/nifti/SABR_new/'

url = url_SABR + patID_l 
scanDir = os.listdir(url)

scan_info = pd.read_csv('D:/data/Aaron/ProstateMRL/Data/MRLPacks/ScanInfo/' + patID + '.csv')
scan_info = scan_info.dropna()

output = 'D:/data/Aaron/ProstateMRL/Data/Extraction/MRLPacks/MeanValues/' 

print("Patient Directory: " + url)
print("Output Directory: " + output)

#print(scan_info.head())


In [ ]:
def MaskedImage(image_url, mask_url):
    image = sitk.ReadImage(image_url)
    image_array = sitk.GetArrayFromImage(image)
    
    mask = sitk.ReadImage(mask_url)
    mask_array = sitk.GetArrayFromImage(mask) 
    if "glute" in mask_url:
        mask_array = mask_array / 13
    
    if image_array.shape != mask_array.shape:
       # print(image_url, mask_url)
       # print(image_array.shape, mask_array.shape)
        mean = np.nan
        std = np.nan
    
    else:
        masked_image = ma.masked_array(image_array, mask=np.logical_not(mask_array), keep_mask=True, hard_mask=True)
        mean = np.mean(masked_image.flatten())
        std = np.std(masked_image.flatten())


    return mean 
    



In [9]:
def GetNorm(image_name): 
    n = image_name

    if "reg" in n:
        Norm = "Raw"
    elif "pros" in n:
        Norm = "Norm-Pros"
    elif "glute" in n:
        Norm = "Norm-Glute"
    elif "psoas" in n:
        Norm = "Norm-Psoas"
    elif "TP" in n:
        Norm = "HM-TP"
    elif "FS" in n:
        Norm = "HM-FS"
    else:
        Norm = "-"
    
    return Norm

def GetRegion(mask_name):
    n = mask_name

    if "pros" in n:
        Region = "Prostate"
    elif "glute" in n: 
        Region = "Glute"
    elif "psoas" in n:
        Region = "Psoas"
    else:
        Region = "-"
    
    return Region



In [14]:
test = "D:\\data\\prostateMR_radiomics\\nifti\\20fractions\\"
print(test.split("\\")[4])

20fractions


In [40]:
df = pd.read_csv("D:\\data\\Aaron\\ProstateMRL\\Data\\MRLPacks\\InterFractionChanges\\0000752.csv")
df.loc[(df.Region == "Prostate") & (df.Normalisation == "Raw") & (df.Fraction == int(1))]["MeanSignal"].iloc[0]

47.206125206425455

In [20]:
test = {"Raw": {"Pros": 100, "Glute": 50},
        "Norm": {"Pros": 10, "Glute": 5}}
region = "Glute"
norm = "Raw"
print(test[norm][region])

50


In [ ]:
results_df = pd.DataFrame(columns=["PatID", "MRCont", "Fraction","ScanDate", "ScanTime", "Scan", "Region", "MeanSignal", "TimeDiff","SignalDiff"])
scanValues = {"PatID":[], "MRCont":[], "Fraction":[], "ScanDate":[], "ScanTime":[], "Scan":[], "Region": [], "MeanSignal":[], "TimeDiff":[], "SignalDiff":[]}

In [ ]:
fractions = scan_info.Fraction.unique()
MRs = scan_info.MRContour.unique()
#print(fractions)
for i in fractions:
    #print(i)
    MR = MRs[i].replace(' ', '')
    fraction = fractions[i]

    temp_df = scan_info.loc[scan_info['Fraction'] == fractions[i]]
    #temp_df = scan_info.loc[scan_info['Fraction'] == fractions[i]]
    print('Pat: ' + patID + ' Fraction: ' + str(fraction) + ' MR: ' + MR)

    scanValues["PatID"] = patID
    scanValues["MRCont"] = MR
    scanValues["Fraction"] = fraction
    
    scanValues["ScanDate"] = temp_df.ContourDate.unique()[0]
    scantimes = temp_df.ScanTime.unique()

    scanValues["Observer"] = "RP"
    scanValues["Region"] = "Prostate" # change when add in more

    
    numberofscans = -1
    for j in os.scandir(url + '/' + MR):
        #print(j.name)
        file = j.name
        glute_url = url + '/' + MR + '/' + patID_l + '_' + MR + '_glute.nii'
        pros_url = url + '/' + MR + '/' + patID_l + '_' + MR + '_shrunk_pros.nii'
        psoas_url = url + '/' + MR + '/' + patID_l + '_' + MR + '_psoas.nii'

        mask_urls = [pros_url, glute_url, psoas_url]

        s = -1
        if "reg" in file:
            s = s + 1
            scan = file.split("_")
            scan = scan[4]
            scan = int(scan[0:1])
            scanValues["Scan"] = scan

            scanValues["ScanTime"] = scantimes[numberofscans]

            image_url = url + '/' + MR + '/' + file
            for k in range(len(mask_urls)):
                mean = MaskedImage(image_url, mask_urls[k])
                scanValues["Mean"] = mean
                if k == 0:
                    print("Prostate: " + mask_urls[k])
                    scanValues["Region"] = "Prostate"
                elif k == 1:
                    print("Glute")
                    scanValues["Region"] = "Glute"
                elif k == 2:
                    scanValues["Region"] = "Psoas"

                if scan == 1:
                    firstmean = mean
                    scanValues["SignalDiff"] = 0
                else:
                    scanValues["SignalDiff"] = mean - firstmean
                #print(file)
                #print(" Pros: " + str(pros_image))
                results_df = results_df.append(scanValues, ignore_index=True)

                 



In [ ]:
df = pd.read_csv('D:/data/Aaron/ProstateMRL/Data/MRLPacks/MeanValues/0001088.csv')

